# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 1:30PM,240871,20,SSNA-2173706715,Sartorius Stedim North America,Released
1,Jun 30 2022 1:21PM,240870,20,ALUR020992707,Alumier Labs Inc.,Released
2,Jun 30 2022 1:21PM,240870,20,ALUR663785343,Alumier Labs Inc.,Released
3,Jun 30 2022 1:21PM,240870,20,ALUR118973578,Alumier Labs Inc.,Released
4,Jun 30 2022 1:21PM,240870,20,ALUR638034807,Alumier Labs Inc.,Released
5,Jun 30 2022 1:21PM,240870,20,ALUR268624807,Alumier Labs Inc.,Released
6,Jun 30 2022 1:21PM,240870,20,ALUR606420389,Alumier Labs Inc.,Released
7,Jun 30 2022 1:21PM,240870,20,ALUR965774923,Alumier Labs Inc.,Released
8,Jun 30 2022 1:21PM,240870,20,ALUR611050864,Alumier Labs Inc.,Released
9,Jun 30 2022 12:48PM,240868,10,Enova-10223,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,240866,Released,1
29,240868,Executing,1
30,240868,Released,3
31,240870,Released,8
32,240871,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240861,NaN,NaN,1.0
240866,NaN,NaN,1.0
240868,NaN,1.0,3.0
240870,NaN,NaN,8.0
240871,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240768,0.0,0.0,1.0
240774,0.0,1.0,0.0
240781,0.0,0.0,1.0
240784,0.0,0.0,1.0
240786,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240768,0,0,1
240774,0,1,0
240781,0,0,1
240784,0,0,1
240786,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240768,0,0,1
1,240774,0,1,0
2,240781,0,0,1
3,240784,0,0,1
4,240786,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240768,,,1
1,240774,,1,
2,240781,,,1
3,240784,,,1
4,240786,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.
9,Jun 30 2022 12:48PM,240868,10,Emerginnova
13,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc."
14,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.
15,Jun 30 2022 12:25PM,240860,10,"Citieffe, Inc."
16,Jun 30 2022 12:19PM,240859,20,"ACI Healthcare USA, Inc."
21,Jun 30 2022 12:12PM,240856,10,Bio-PRF
33,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc."
62,Jun 30 2022 11:44AM,240846,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,,,1
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,,,8
2,Jun 30 2022 12:48PM,240868,10,Emerginnova,,1,3
3,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc.",,,1
4,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,,,1
5,Jun 30 2022 12:25PM,240860,10,"Citieffe, Inc.",,,1
6,Jun 30 2022 12:19PM,240859,20,"ACI Healthcare USA, Inc.",,,5
7,Jun 30 2022 12:12PM,240856,10,Bio-PRF,,10,2
8,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc.",,20,9
9,Jun 30 2022 11:44AM,240846,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1,,
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8,,
2,Jun 30 2022 12:48PM,240868,10,Emerginnova,3,1,
3,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc.",1,,
4,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,1,,
5,Jun 30 2022 12:25PM,240860,10,"Citieffe, Inc.",1,,
6,Jun 30 2022 12:19PM,240859,20,"ACI Healthcare USA, Inc.",5,,
7,Jun 30 2022 12:12PM,240856,10,Bio-PRF,2,10,
8,Jun 30 2022 12:03PM,240847,15,"Person & Covey, Inc.",9,20,
9,Jun 30 2022 11:44AM,240846,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1,,
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8,,
2,Jun 30 2022 12:48PM,240868,10,Emerginnova,3,1,
3,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc.",1,,
4,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1.0,NaN,NaN
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8.0,NaN,NaN
2,Jun 30 2022 12:48PM,240868,10,Emerginnova,3.0,1.0,NaN
3,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1.0,0.0,0.0
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8.0,0.0,0.0
2,Jun 30 2022 12:48PM,240868,10,Emerginnova,3.0,1.0,0.0
3,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1204284,8.0,11.0,0.0
15,240847,9.0,20.0,0.0
16,722484,3.0,0.0,0.0
17,240768,1.0,0.0,0.0
18,240813,1.0,0.0,0.0
19,722435,5.0,35.0,18.0
20,1445111,32.0,10.0,0.0
21,963125,3.0,1.0,0.0
22,240846,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1204284,8.0,11.0,0.0
1,15,240847,9.0,20.0,0.0
2,16,722484,3.0,0.0,0.0
3,17,240768,1.0,0.0,0.0
4,18,240813,1.0,0.0,0.0
5,19,722435,5.0,35.0,18.0
6,20,1445111,32.0,10.0,0.0
7,21,963125,3.0,1.0,0.0
8,22,240846,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,11.0,0.0
1,15,9.0,20.0,0.0
2,16,3.0,0.0,0.0
3,17,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,5.0,35.0,18.0
6,20,32.0,10.0,0.0
7,21,3.0,1.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,15,Released,9.0
2,16,Released,3.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
Executing,11.0,20.0,0.0,0.0,0.0,35.0,10.0,1.0,0.0
Released,8.0,9.0,3.0,1.0,1.0,5.0,32.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
1,Executing,11.0,20.0,0.0,0.0,0.0,35.0,10.0,1.0,0.0
2,Released,8.0,9.0,3.0,1.0,1.0,5.0,32.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
1,Executing,11.0,20.0,0.0,0.0,0.0,35.0,10.0,1.0,0.0
2,Released,8.0,9.0,3.0,1.0,1.0,5.0,32.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()